In [ ]:
import requests
import pandas as pd

# Foursquare API token
fsq_api_token = "fsq3Rg6JOtHbwU3x591rWCjbFdipW+f/h/WhItjUfbbsiNc="

# Yelp API token
yelp_api_token = "AA4otC-u9QV-D1CkpInkb7HiFjKaSZjkOxoMmgJezfN--fAIHjyfCHURr7lL5HkefvOqcNKMYewF0y7ihDjOmcTj_R7FKTHzfvE0eYNBiDYKVxKGvyXXT4KkM9jsZHYx"

# Load bike stations data
bike_stations_df = pd.read_csv('divvy_station_data.csv')

# Lists to store combined data
combined_fsq_data = []
combined_yelp_data = []

# Loop through each bike station
for index, row in bike_stations_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Foursquare API request
    fsq_url = f"https://api.foursquare.com/v2/venues/search?ll={latitude},{longitude}&categoryId=4d4b7104d754a06370d81259,4d4b7105d754a06374d81259&oauth_token={fsq_api_token}&v=20230815"
    fsq_response = requests.get(fsq_url)
    fsq_results = fsq_response.json().get("response", {}).get("venues", [])
    fsq_pois = []
    for poi in fsq_results:
        categories = [category['name'] for category in poi['categories']]
        fsq_pois.append({
            "Name": poi['name'],
            "Categories": ", ".join(categories),
            "Latitude": poi['location']['lat'],
            "Longitude": poi['location']['lng']
        })
    
    # Yelp API request
    yelp_url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&categories=restaurant,bar,entertainment&sort_by=best_match&limit=20"
    yelp_headers = {
        "Authorization": f"Bearer {yelp_api_token}",
        "accept": "application/json"
    }
    yelp_response = requests.get(yelp_url, headers=yelp_headers)
    yelp_results = yelp_response.json().get("businesses", [])
    yelp_pois = []
    for poi in yelp_results:
        categories = [category['title'] for category in poi['categories']]
        yelp_pois.append({
            "Name": poi['name'],
            "Categories": ", ".join(categories),
            "Latitude": poi['coordinates']['latitude'],
            "Longitude": poi['coordinates']['longitude']
        })
    
    combined_fsq_data.append({
        "Station": row['Station'],
        "Latitude": latitude,
        "Longitude": longitude,
        "POIs": fsq_pois
    })
    
    combined_yelp_data.append({
        "Station": row['Station'],
        "Latitude": latitude,
        "Longitude": longitude,
        "POIs": yelp_pois
    })

# Create DataFrames
combined_fsq_df = pd.DataFrame(combined_fsq_data)
combined_yelp_df = pd.DataFrame(combined_yelp_data)

# Save DataFrames to CSV
combined_fsq_df.to_csv('combined_foursquare_data.csv', index=False)
combined_yelp_df.to_csv('combined_yelp_data.csv', index=False)

print("Combined Foursquare data saved to 'combined_foursquare_data.csv'")
print("Combined Yelp data saved to 'combined_yelp_data.csv'")
